# Create a meshed screen with a plane-wave beam

The screen is placed at the origin with the normal pointing (1/sqrt(2), 0, -1/sqrt(2)) <br>
The beam is propagating in positive z direction and would be reflected to positive x direction (left).<br>
The beam is a plane wave but with a gaussian temporal pulse shape.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import constants
from MeshedFields import *

In [2]:
mesh = MeshedField.CircularMesh(R=0.01, ratio=10.0, lcar=0.2)
print("%d points" % len(mesh.points))
print("%d triangles" % len(mesh.triangles))

1155 points
2100 triangles


In [3]:
pts = [np.array([p[0],p[1],p[0]])for p in mesh.points]
source = MeshedField(pts, mesh.triangles)

In [4]:
area = source.MeshArea()
print(np.sum(area))

0.00044401437102


In [5]:
normals = source.MeshNormals()
print(np.sum(normals, axis=0))

[ 1484.92424049     0.         -1484.92424049]


### define the beam parameters

In [24]:
# time step
dt=1.0e-13
NOTS=400

# wave parameters
tau = 5.0e-12
f = 0.2e12
print("f = %.2f THz" % (f*1e-12))
lam = constants.c/f
print("λ = %.2f µm" % (lam*1e6))
kz = 2.0*np.pi/lam
print("k = %.2f 1/cm" % (kz*1e-2))

f = 0.20 THz
λ = 1498.96 µm
k = 41.92 1/cm


### compute the field traces for every point

In [25]:
# all points use the same timing grid
# the center time of each trace varies along with the z coordinate
t = np.linspace(-NOTS/2*dt, (NOTS/2-1)*dt, NOTS)
source.dt = dt
source.Nt = NOTS
# the time traces start such that the peak of the wave passes at the center time
source.t0 = [p[2]/constants.c - NOTS/2*dt for p in source.pos]

# field (6 components) traces on screen
source.A = np.zeros((source.Np,NOTS,6))
# electric field amplitude in V/m
E0 = 2.0e7
# magnetic field amplitude in T
B0 = E0 / constants.c

for ip, p in enumerate(source.pos):
    z = p[2]
    # field strength
    time = t + source.t0[ip]
    osc = np.cos((2.0*np.pi*f*time)-kz*z) * np.exp(-np.square(t)/(2.0*tau*tau))
    Ex = E0 * osc
    Ey = np.zeros_like(Ex)
    Ez = np.zeros_like(Ex)
    Bx = np.zeros_like(Ex)
    By = B0 * osc
    Bz = np.zeros_like(Ex)
    trace = np.array([Ex,Ey,Ez,Bx,By,Bz]).transpose()
    source.A[ip] = trace

In [52]:
filename="PlaneWave.h5"
source.WriteMeshedField(filename)

In [53]:
# compute power density for plotting
area = source.MeshArea()
P = source.EnergyFlowDensity()
Pz = P[:,2]

print("total mesh area = %7.3f cm²" % (1.0e4*np.sum(area)))
print("peak energy density = %.6f J/m²" % np.max(Pz))
print("total pulse energy = %.1f µJ" % (1e6*np.dot(area,Pz)))

total mesh area =   4.440 cm²
peak energy density = 4.704835 J/m²
total pulse energy = 2089.0 µJ


In [54]:
source.ShowMeshedField(scalars=Pz,scalarTitle="Pz",showCenters=False)

In [41]:
Ex0 = source.FieldAtTime(-250.0*dt)[:,0]
source.ShowMeshedField(scalars=Ex0,scalarTitle="Ex",showCenters=False,lut=phaseLUT())

In [44]:
# this forces a separate plot window
%matplotlib qt

# plot a field trace when a cell is clicked
# subsequent plots are overlaid in the same window
# the plot window only becomes active when the VTK window is closed (it should be spawned as a separate thread)
def action(id):
    print(source.pos[id])
    print(source.t0[id])
    print(source.t0[id]+NOTS/2*dt)
    source.ShowFieldTrace(id)

Ex0 = source.FieldAtTime(0.0*dt)[:,0]
source.ShowMeshedField(scalars=Ex0,scalarTitle="Ex",showCenters=False,lut=phaseLUT(),pickAction=action)

[ 0.00848194 -0.00124587  0.00848194]
8.29270884959e-12
2.82927088496e-11
[ 0.00059456 -0.00070818  0.00059456]
-1.80167741859e-11
1.98322581407e-12
[ 0.0008057  -0.00053953  0.0008057 ]
-1.73124809563e-11
2.68751904366e-12


In [42]:
Ex0 = source.FieldAtTime(250.0*dt)[:,0]
source.ShowMeshedField(scalars=Ex0,scalarTitle="Ex",showCenters=False,lut=phaseLUT())

In [51]:
print(source.pos[1509])
print(source.t0[1509])
source.ShowFieldTrace(1509)
print(source.pos[1504])
print(source.t0[1504])
source.ShowFieldTrace(1504)

[ 0.00041569 -0.0006844   0.00041569]
-1.86134198855e-11
[ 0.00074974 -0.00142461  0.00074974]
-1.74991339286e-11
